# Gathering and Sanitizing Netflix User Data

A concern when sharing information online is giving out more information than might be safe. Since this report is going to be about Netflix profile watch data the chances of that being true here is small, but not null. After we pull the data we will check the information included in the base file, then sanitized and/or [hash](https://www.techtarget.com/searchdatamanagement/definition/hashing)  the data.

To download the usage information from Netflix navigate to the url `https://www.netflix.com/account/getmyinfo` and submit a request for a copy of your information. After a short period of time you will receive an email from Netflix with a CSV file of your data.

Download and unzip the file. Inside are 11 folders and two files:

![netflix-report-tree structure](./files/assets/netflix-report-tree.png 'netflix report tree')

## Viewing Activity

The first report that we are looking for is `./CONTENT_INTERACTION/ViewingActivity.csv` and we can move that into the `files/non-sanitized` folder

We can now pull in the data and check what the columns are.

In [1]:
import pandas as pd

file_dir = r'./files'
non_sanitized_dir = fr'{file_dir}/non-sanitized'

df_v = pd.read_csv(fr'{non_sanitized_dir}/ViewingActivity.csv')
df_v.dtypes

Duration                   object
Start Time                 object
Profile Name               object
Country                    object
Bookmark                   object
Latest Bookmark            object
Supplemental Video Type    object
Attributes                 object
Device Type                object
Title                      object
dtype: object

We won't need all of these columns to calulate how many Hours were Watched in a month. So lets look at each column to see if it holds data we want to keep.

A full description of each column can be found on page 11 of the PDF [`Cover Sheet.pdf`](./files/Cover%20Sheet.pdf) from the root folder downloaded from Netflix. A summary of the data is as follows:

|Column Name | Description |
| --- | --- |
| Profile Name | The profile which viewed the program |
| Start Time | The UTC date and time viewing started. |
| Duration | Length of the viewing session |
| Attributes | Shows additional details of interactions with streamed content, where available. |
| Title | The Program viewed. |
| Supplemental Video Type | Videos other than a TV show or movie, such as trailers or montages |
| Device type | Device type from which the program was streamed |
| Bookmark | Most recent viewing position from the playback session of the program. |
| Latest Bookmark | Indicates whether the Bookmark is the most recent viewing postions. *Not latest view* indicates that it is not the most recent playback |
| Country | Country from which the program was viewed. |

Note: The term *program* can mean either a TV show or a movie.


The only columns that we need to calculate our number are `Start Time`, and `Duration`, however under the description for `Supplemental Video Type` we can see that some of the views are not for programs themselves but rather their trailers/hooks/etc. When we create the sanitized dataset we can also filter out all records where the `Supplemental Video Type` is anything other than empty.

In [9]:
df_vc = df_v.loc[df_v['Supplemental Video Type'].isnull()][['Start Time', 'Duration']]
df_vc
df_vc.sample(10)

,Start Time,Duration
8936,2020-09-23 09:40:25,01:12:14
8902,2020-09-28 16:59:54,00:00:37
10210,2020-03-27 12:19:38,00:02:00
13175,2016-12-31 11:25:19,00:21:33
9073,2020-09-13 07:24:48,00:42:30
527,2025-07-11 03:18:44,00:20:03
8453,2020-12-21 09:37:38,00:58:04
1130,2025-05-08 03:42:53,00:20:01
6733,2022-01-06 08:40:59,00:23:22
6310,2022-02-19 14:25:22,00:01:58


Now we save the data. We don't need the index so we will also remove the index column.

To maintain data integrity we save the file in a different location

In [3]:
df_vc.to_csv(rf'{file_dir}/view_activity_by_month.csv', index=False)

## Billing History

The next report can be found in the `./PAYMENT_AND_BILLING/BillingHistory.csv`, and we will place in the same `files/non-sanitized` folder

In [4]:
df_b = pd.read_csv(fr'{non_sanitized_dir}/BillingHistory.csv')
df_b.dtypes

Transaction Date              object
Country                       object
Mop Last 4                     int64
Final Invoice Result          object
Mop Pmt Processor Desc        object
Pmt Txn Type                  object
Description                   object
Gross Sale Amt               float64
Pmt Status                    object
Payment Type                  object
Tax Amt                      float64
Service Period Start Date     object
Item Price Amt               float64
Mop Creation Date             object
Currency                      object
Next Billing Date             object
Service Period End Date       object
dtype: object

We won't need all of these columns to get the subscription cost. Lets again look at each column to see if it holds data we want to keep.

The report that we are using is `BillingHistory.csv`. In the data downloaded from Netflix, in the PDF [`Cover Sheet.pdf`](./files/Cover%20Sheet.pdf) on page 16 we can find a full description of the data that each column holds. A summary of the data is as follows:

|Column Name | Description |
| --- | --- |
| `Transaction Date` | Date the payment was charged |
| `Service Period Start Date` | Start date of the subscription period |
| `Service Period End Date` | End date of the subscription period |
| `Description` |  Type of charge incurred. ex:, "subscription", is the cost for the subscription; "upgrade" means that the subscription was upgraded |
| `Payment Type` | Payment associated with your account<br>*note: removed for this report* |
| `Mop Last 4` | last four digits of your method of payment<br>*note: removed for this report*|
| `Mop Creation Date` | Date the mop was added |
| `Mop Pmt Processor Desc` | Processor used for the payment transaction |
| `Item Price Amt` | Cost of subscription, without tax |
| `Currency` | Currency in which method of payment is charged |
| `Tax Amt` | Tax amount applied to `Item Price Amt`. |
| `Gross Sale Amt` | Total amount charged for subscription, including tax. |
| `Pmt Txn Type` | Stage that payment transaction has entered. ex:<br>`SALE`: renewal began;<br>`CAPTURED`: payment captured by `Mop Pmt Processor`. |
| `Pmt Status` | Status of the payment transaction. ex: <br>`declined`: transaction was declined; <br>`approved` transaction was approved and the mop was charged; <br>`pending`:  transaction is pending decline or approval. |
| `Final Invoice Result` | Charges made or attempted to make toward account. ex: <br>`COMPLETED` Charge was successful;<br>`SETTLED`: The fee has reached Netflix; <br> `AUTHORIZATIONS`: Requests to verify that the mop works; <br>`APPLIED` Gift card applied to account; `DEDUCTED`: charge deducted from the balance of the gift card |
| `Country` | Country in which the account subscription is initiated.<br>*note: removed for this report* |
| `Next Billing Date` | Next date on which the member will be billed. |


Note: The term *mop* means Method of Payment.

While the only columns that we need for the calculations are `Service Period Start Date` and `Gross Sale Amt` this report includes multiple records for each month due to the multiple 'handshakes' that a merchant and bank perform when making a purchase. For some of these the 'Service Period Start Date' value is null and will need to be filtered out. Before making the changes we want to make sure that we only remove the data that needs to be removed.

In [5]:
df_b = df_b.loc[df_b['Service Period Start Date'].notnull()]

b_tm = len(df_b['Service Period Start Date'].unique())
b_vc = df_b['Service Period Start Date'].value_counts()

print(f'total months: {b_tm}\nmonths with multiple records: {len(b_vc[b_vc>1])}')

total months: 89
months with multiple records: 51


By leaving only these two columns we also have removed the markers that made the additional records unique, which then allows us to usilize the built-in `.drop_duplicates()` function to clear those records.

In [6]:
df_bpm = df_b[['Service Period Start Date', 'Gross Sale Amt']].drop_duplicates()

bpm_tm = len(df_b['Service Period Start Date'].unique())
bpm_vc = df_bpm['Service Period Start Date'].value_counts()

print(f'total months: {bpm_tm}\nmonths with multiple records: {len(bpm_vc[bpm_vc>1])}')
df_bpm.sample(n=5)

total months: 89
months with multiple records: 0


,Service Period Start Date,Gross Sale Amt
138,2022-09-02,15.49
86,2023-10-01,19.99
60,2024-04-01,22.99
161,2022-04-02,15.49
70,2024-02-01,22.99


we can now save the new report for later use.

In [10]:
df_bpm.to_csv(rf'{file_dir}/billing_by_month.csv', index=False)